# Twisted $L^2$-Euler characteristic of the Borromean rings complement

In [1]:
import itertools
import twisted_l2
import regina
import snappy

## Compute the chain complex of the universal cover:

In [2]:
twisted_l2.load_hap()

true

In [3]:
mfd = snappy.Manifold("L6a4")
tri = regina.Triangulation3(mfd)
print(tri.fVector())

[3, 8, 16, 8]

In [4]:
tri.idealToFinite()

True

In [5]:
tri.fVector()

[40, 280, 464, 224]

In [6]:
%time fl = twisted_l2.regina_tri_to_face_lattice(tri, ideal=False, simplify=False)

CPU times: user 52.6 ms, sys: 3.27 ms, total: 55.8 ms
Wall time: 55.5 ms


In [7]:
%time cw = twisted_l2.cw_complex(fl)

CPU times: user 65.7 ms, sys: 137 µs, total: 65.8 ms
Wall time: 65.7 ms


In [8]:
number_of_cells = twisted_l2.gap_member(cw, "nrCells")
print("Number of cells in each dimension:")
for j in range(6):
    print(f"{j}: {number_of_cells(j)}")

Number of cells in each dimension:
0: 1008
1: 6480
2: 10848
3: 5376
4: 0
5: 0


In [9]:
%time chain_complex = twisted_l2.equivariant_cc(cw, gap=False)

CPU times: user 140 ms, sys: 144 µs, total: 140 ms
Wall time: 140 ms


In [10]:
[chain_complex.dimension(i) for i in range(5)]

[1, 3, 2, 0, 0]

In [11]:
G = twisted_l2.get_fundamental_group(chain_complex)

In [12]:
cc = twisted_l2.get_differentials(chain_complex)

The variable `chain_complex` wraps a GAP object,
while `cc` is simply a list of matrices over the group algebra of `G`. \
Actually, for technical reasons, we use the free group on the generators of `G` instead of `G` itself.

The above cell is equivalent to
```
cc = [twisted_l2.boundary_operator(chain_complex, i, G) for i in range(1, 3)]
```

In [13]:
G

Finitely presented group < f1, f2, f3 | f3^-1*f2^-1*f3*f1*f3^-1*f1^-1*f2*f1*f3*f1^-1, f2^-1*f1*f2*f3^-1*f2^-1*f3*f1^-1*f3^-1*f2*f3 >

## Or load precomputed data:

We can also save/load the group and chain complex.\
This is useful to deal with non-deterministic results from CW complex simplifications etc.

In [14]:
# twisted_l2.save_to_file("borromean-2.json", group=G, cc=cc)
G, cc = twisted_l2.load_from_file("borromean.json")

In [15]:
G.abelian_invariants()

(0, 0, 0)

In [16]:
G

Finitely presented group < x0, x1, x2 | x2^-1*x1^-1*x2*x0*x2^-1*x0^-1*x1*x0*x2*x0^-1, x1^-1*x0*x1*x2^-1*x1^-1*x2*x0^-1*x2^-1*x1*x2 >

$G$ has rank-$3$ abelianization, so the unit ball of the Thurston norm lives in $3$-space.

In [17]:
# Choose the desired configuration (see "configs.py")
twisted_l2.configs.LogOptions.LEVEL = twisted_l2.MINIMAL
twisted_l2.configs.LogOptions.PRECISION = 8

## Convergence of the algorithm

First, we investigate what happens with different finite quotients (controlled by the argument `exps`).\
We use a single $\phi \in \mathrm{Hom}(G; \mathbb Z)$ represented by `v = (0,0,1)`.

In [18]:
twisted_l2.make_phi_from_coordinates(G, [0,0,1], as_list=True)

[0, 0, 1]

These are the values of $\phi(x_0), \phi(x_1), \phi(x_2)$, where $x_0, x_1, x_2$ are the generators of $G$.

We first run the algorithm with a high expansion parameter such as $10$,\
and we see that the (computed) valuations are all bounded above by $6$.

In [19]:
for i in range(4):
    print(f"===== With finite {Primes().unrank(i)}-quotient =====")
    exps = [0] * i + [1]
    twisted_l2.characteristic(G, (0,0,1), cc, 10, exps)
    print()

===== With finite 2-quotient =====
Degrees of Laplacians : [2.0, -6.0, -4.0]
Valuations            : [0.0, 6.0, 4.0]
Quotient sizes        : [4, 4, 4]
Characteristic        : 1.0

===== With finite 3-quotient =====
Degrees of Laplacians : [2.0, -2.0, -1.33333333]
Valuations            : [0.0, 4.0, 2.66666667]
Quotient sizes        : [9, 9, 9]
Characteristic        : 0.33333333

===== With finite 5-quotient =====
Degrees of Laplacians : [2.0, 1.2, 0.8]
Valuations            : [0.0, 2.4, 1.6]
Quotient sizes        : [25, 25, 25]
Characteristic        : -0.2

===== With finite 7-quotient =====
Degrees of Laplacians : [2.0, 2.57142857, 1.71428571]
Valuations            : [0.0, 1.71428571, 1.14285714]
Quotient sizes        : [49, 49, 49]
Characteristic        : -0.42857143



Hence, by heuristically taking expansion parameter $6$, we try doing it with more primes (up to $29$).

In [21]:
for i in range(10):
    print(f"===== With finite {Primes().unrank(i)}-quotient =====")
    exps = [0] * i + [1]
    twisted_l2.characteristic(G, (0,0,1), cc, 6, exps)
    print()

===== With finite 2-quotient =====
Degrees of Laplacians : [2.0, -6.0, -4.0]
Valuations            : [0.0, 6.0, 4.0]
Quotient sizes        : [4, 4, 4]
Characteristic        : 1.0

===== With finite 3-quotient =====
Degrees of Laplacians : [2.0, -2.0, -1.33333333]
Valuations            : [0.0, 4.0, 2.66666667]
Quotient sizes        : [9, 9, 9]
Characteristic        : 0.33333333

===== With finite 5-quotient =====
Degrees of Laplacians : [2.0, 1.2, 0.8]
Valuations            : [0.0, 2.4, 1.6]
Quotient sizes        : [25, 25, 25]
Characteristic        : -0.2

===== With finite 7-quotient =====
Degrees of Laplacians : [2.0, 2.57142857, 1.71428571]
Valuations            : [0.0, 1.71428571, 1.14285714]
Quotient sizes        : [49, 49, 49]
Characteristic        : -0.42857143

===== With finite 11-quotient =====
Degrees of Laplacians : [2.0, 3.81818182, 2.54545455]
Valuations            : [0.0, 1.09090909, 0.72727273]
Quotient sizes        : [121, 121, 121]
Characteristic        : -0.63636364


Two observations about the above runs:

1. the size of the finite quotient $L$ is $p^2$
2. the valuations are $[0, \frac{12}{p}, \frac{8}{p}]$

Valuations appear to tend to zero as $|L|^{-1/2}$.\
If we heuristically take $[0,0,0]$ as the exact values, the corresponding degrees of the Laplacians are $[2,6,4]$, so we infer

$$\chi^{(2)}(\tilde{M}; \phi) = -\frac{1}{2} (0\cdot 2 - 1\cdot 6 + 2\cdot 4) = -1.$$

The exact same observations hold for the product of two such quotients (obtained from different primes):

In [8]:
%%time
for i in range(10):
    for j in range(i+1, 10):
        p = Primes().unrank(i)
        q = Primes().unrank(j)
        if p * q > 33:
            continue
        print(f"===== {p}-quotient x {q}-quotient =====")
        exps = [0] * i + [1] + [0] * (j-i-1) + [1]
        twisted_l2.characteristic(G, (0,0,1), cc, 6, exps)
        print()

===== 2-quotient x 3-quotient =====
Degrees of Laplacians : [2.0, 2.0, 1.33333333]
Valuations            : [0.0, 2.0, 1.33333333]
Quotient sizes        : [36, 36, 36]
Characteristic        : -0.33333333

===== 2-quotient x 5-quotient =====
Degrees of Laplacians : [2.0, 3.6, 2.4]
Valuations            : [0.0, 1.2, 0.8]
Quotient sizes        : [100, 100, 100]
Characteristic        : -0.6

===== 2-quotient x 7-quotient =====
Degrees of Laplacians : [2.0, 4.28571429, 2.85714286]
Valuations            : [0.0, 0.85714286, 0.57142857]
Quotient sizes        : [196, 196, 196]
Characteristic        : -0.71428571

===== 2-quotient x 11-quotient =====
Degrees of Laplacians : [2.0, 4.90909091, 3.27272727]
Valuations            : [0.0, 0.54545455, 0.36363636]
Quotient sizes        : [484, 484, 484]
Characteristic        : -0.81818182

===== 2-quotient x 13-quotient =====
Degrees of Laplacians : [2.0, 5.07692308, 3.38461538]
Valuations            : [0.0, 0.46153846, 0.30769231]
Quotient sizes        

In hindsight, since the true values of the valuations appear to be $[0,0,0]$,\
we could have chosen expansion parameter `n = 1`, which is much faster.

Let's investigate what happens with more complicated quotients and `n = 1`.\
First, we look at products of $1$-nilpotent quotients (where max(`exps`) is $1$).

In [23]:
%%time
for i in range(11):
    for j in range(i+1, 11):
        p = Primes().unrank(i)
        q = Primes().unrank(j)
        if p * q > 70:
            continue
        print(f"===== {p}-quotient x {q}-quotient =====")
        exps = [0] * i + [1] + [0] * (j-i-1) + [1]
        twisted_l2.characteristic(G, (0,0,1), cc, 1, exps)
        print()

===== 2-quotient x 3-quotient =====
Degrees of Laplacians : [2.0, 5.33333333, 3.33333333]
Valuations            : [0.0, 0.33333333, 0.33333333]
Quotient sizes        : [1, 36, 36]
Characteristic        : -0.66666667

===== 2-quotient x 5-quotient =====
Degrees of Laplacians : [2.0, 5.6, 3.6]
Valuations            : [0.0, 0.2, 0.2]
Quotient sizes        : [1, 100, 100]
Characteristic        : -0.8

===== 2-quotient x 7-quotient =====
Degrees of Laplacians : [2.0, 5.71428571, 3.71428571]
Valuations            : [0.0, 0.14285714, 0.14285714]
Quotient sizes        : [1, 196, 196]
Characteristic        : -0.85714286

===== 2-quotient x 11-quotient =====
Degrees of Laplacians : [2.0, 5.81818182, 3.81818182]
Valuations            : [0.0, 0.09090909, 0.09090909]
Quotient sizes        : [1, 484, 484]
Characteristic        : -0.90909091

===== 2-quotient x 13-quotient =====
Degrees of Laplacians : [2.0, 5.84615385, 3.84615385]
Valuations            : [0.0, 0.07692308, 0.07692308]
Quotient sizes 

In [24]:
%%time
for exps in [(1,1,1), (1,1,0,1), (1,1,0,0,1), (1,0,1,1)]:
    description_string = ', '.join(str(Primes().unrank(j)) for j,c in enumerate(exps) if c)
    print(f"===== Product of p-quotients, p = {description_string} =====")
    twisted_l2.characteristic(G, (0,0,1), cc, 1, exps)
    print()

===== Product of p-quotients, p = 2, 3, 5 =====
Degrees of Laplacians : [2.0, 5.86666667, 3.86666667]
Valuations            : [0.0, 0.06666667, 0.06666667]
Quotient sizes        : [1, 900, 900]
Characteristic        : -0.93333333

===== Product of p-quotients, p = 2, 3, 7 =====
Degrees of Laplacians : [2.0, 5.9047619, 3.9047619]
Valuations            : [0.0, 0.04761905, 0.04761905]
Quotient sizes        : [1, 1764, 1764]
Characteristic        : -0.95238095

===== Product of p-quotients, p = 2, 3, 11 =====
Degrees of Laplacians : [2.0, 5.93939394, 3.93939394]
Valuations            : [0.0, 0.03030303, 0.03030303]
Quotient sizes        : [1, 4356, 4356]
Characteristic        : -0.96969697

===== Product of p-quotients, p = 2, 5, 7 =====
Degrees of Laplacians : [2.0, 5.94285714, 3.94285714]
Valuations            : [0.0, 0.02857143, 0.02857143]
Quotient sizes        : [1, 4900, 4900]
Characteristic        : -0.97142857

CPU times: user 3min 58s, sys: 23.3 ms, total: 3min 58s
Wall time: 3min

In general, if $P$ is the product of the primes, we have $|L| = P^2$,
except for the zeroth Laplacian, for which we obtain a trivial quotient.\
This is not a problem, because $\Delta_0$ is a $1\times 1$ matrix,
and it is easy to see that its $\phi$-degree is really $2$:

In [25]:
twisted_l2.ga_adjoint(cc[0]) * cc[0]

[6 - F[0]^-1 - F[0] - F[1]^-1 - F[1] - F[2]^-1 - F[2]]

Indeed, its Dieudonné determinant is represented by its only entry, whose Newton polytope is an octahedron of thickness $2$ in the $(0,0,1)$ direction.

As for the error, it seems to be always $2 / P$. Experimentally, the coefficient $2$ depends on `n`:

|  n  | coeff |
|:---:|:-----:|
|  1  |   2   |
|  2  |   4   |
|  3  |   6   |
|  4  |   8   |
|  5  |   6   |
|  6  |   4   |

It seems that choosing `n` as low as possible, but still higher than the valuations, leads to a tighter estimate by the algorithm.\
The following cells show this dependence for `exps = (1,1,1)` and `exps = (1,1)` (other values can be chosen):

In [30]:
%%time
for j in range(1,7):
    exps = (1,1,1)
    P = product(Primes().unrank(k) for k,c in enumerate(exps) if c)
    print(f"===== n = {j} =====")
    char = twisted_l2.characteristic(G, (0,0,1), cc, j, exps)
    error = char + 1
    print(f"Error                 : {error*P}/{P}")
    print()

===== n = 1 =====
Degrees of Laplacians : [2.0, 5.86666667, 3.86666667]
Valuations            : [0.0, 0.06666667, 0.06666667]
Quotient sizes        : [1, 900, 900]
Characteristic        : -0.93333333
Error                 : 2/30

===== n = 2 =====
Degrees of Laplacians : [2.0, 5.73333333, 3.73333333]
Valuations            : [0.0, 0.13333333, 0.13333333]
Quotient sizes        : [900, 900, 900]
Characteristic        : -0.86666667
Error                 : 4/30

===== n = 3 =====
Degrees of Laplacians : [2.0, 5.6, 3.6]
Valuations            : [0.0, 0.2, 0.2]
Quotient sizes        : [900, 900, 900]
Characteristic        : -0.8
Error                 : 6/30

===== n = 4 =====
Degrees of Laplacians : [2.0, 5.46666667, 3.46666667]
Valuations            : [0.0, 0.26666667, 0.26666667]
Quotient sizes        : [900, 900, 900]
Characteristic        : -0.73333333
Error                 : 8/30

===== n = 5 =====
Degrees of Laplacians : [2.0, 5.33333333, 3.46666667]
Valuations            : [0.0, 0.33333

In [31]:
%%time
for j in range(1,10):
    exps = (1,1)
    P = product(Primes().unrank(k) for k,c in enumerate(exps) if c)
    print(f"===== n = {j} =====")
    char = twisted_l2.characteristic(G, (0,0,1), cc, j, exps)
    error = char + 1
    print(f"Error                 : {error*P}/{P}")
    print()

===== n = 1 =====
Degrees of Laplacians : [2.0, 5.33333333, 3.33333333]
Valuations            : [0.0, 0.33333333, 0.33333333]
Quotient sizes        : [1, 36, 36]
Characteristic        : -0.66666667
Error                 : 2/6

===== n = 2 =====
Degrees of Laplacians : [2.0, 4.66666667, 2.66666667]
Valuations            : [0.0, 0.66666667, 0.66666667]
Quotient sizes        : [36, 36, 36]
Characteristic        : -0.33333333
Error                 : 4/6

===== n = 3 =====
Degrees of Laplacians : [2.0, 4.0, 2.0]
Valuations            : [0.0, 1.0, 1.0]
Quotient sizes        : [36, 36, 36]
Characteristic        : 0.0
Error                 : 6/6

===== n = 4 =====
Degrees of Laplacians : [2.0, 3.33333333, 1.33333333]
Valuations            : [0.0, 1.33333333, 1.33333333]
Quotient sizes        : [36, 36, 36]
Characteristic        : 0.33333333
Error                 : 8/6

===== n = 5 =====
Degrees of Laplacians : [2.0, 2.66666667, 1.33333333]
Valuations            : [0.0, 1.66666667, 1.33333333]


More complicated quotients involve $2$-nilpotent groups:

In [37]:
%%time
for exps in [(2,), (2,1), (2,0,1), (0,2)]:
    description_string = ', '.join(str(c) for c in exps)
    print(f"===== exps = ({description_string}) =====")
    char = twisted_l2.characteristic(G, (0,0,1), cc, 1, exps)
    # retrieve the size of L for the second Laplacian (the first also works)
    Ls = twisted_l2.get_twisted_l2_logs()[2].Lsize.sage()
    print(f"Error * sqrt(|L|)     : {float((char + 1)*sqrt(Ls)):.5f}")
    print()

===== exps = (2) =====
Degrees of Laplacians : [2.0, 5.0, 2.703125]
Valuations            : [0.0, 0.5, 0.6484375]
Quotient sizes        : [1, 128, 128]
Characteristic        : -0.203125
Error * sqrt(|L|)     : 9.01561

===== exps = (2, 1) =====
Degrees of Laplacians : [2.0, 5.66666667, 3.52256944]
Valuations            : [0.0, 0.16666667, 0.23871528]
Quotient sizes        : [1, 1152, 1152]
Characteristic        : -0.68923611
Error * sqrt(|L|)     : 10.54768

===== exps = (2, 0, 1) =====
Degrees of Laplacians : [2.0, 5.8, 3.708125]
Valuations            : [0.0, 0.1, 0.1459375]
Quotient sizes        : [1, 3200, 3200]
Characteristic        : -0.808125
Error * sqrt(|L|)     : 10.85409

===== exps = (0, 2) =====
Degrees of Laplacians : [2.0, 5.55555556, 3.3397348]
Valuations            : [0.0, 0.22222222, 0.3301326]
Quotient sizes        : [1, 2187, 2187]
Characteristic        : -0.56195702
Error * sqrt(|L|)     : 20.48524

CPU times: user 1min 57s, sys: 23.2 ms, total: 1min 57s
Wall time: 

Assuming that these quotients fit the same $O(|L|^{-1/2})$ trend for the errors,\
the coefficients are around $10$ when there's a $2$-nilpotent $2$-group in the product,
and around $20$ when there's a $2$-nilpotent $3$-group.

The simpler max(`exps`) = $1$ case gives better results, which is surprising as the quotients in that case are abelian.\
Abelian quotients give an approximation of the _Alexander norm_, which is not equal to the Thurston norm in general.

For the Borromean rings complement, they _are_ equal, and the unit ball is an octahedron with vertices at permutations of $(\pm 1,0,0)$.\
The norm is determined by the following values of the twisted $L^2$-Euler characteristic:
- $(\pm 1, \phantom{\pm} 0, \phantom{\pm} 0) \mapsto -1$
- $(\phantom{\pm} 0, \pm 1, \phantom{\pm} 0) \mapsto -1$
- $(\phantom{\pm} 0, \phantom{\pm} 0, \pm 1) \mapsto -1$
- $(\pm 1, \pm 1, \pm 1) \mapsto -3$

### The unit ball

Because of this equality of norms, to compute the unit ball, it is acceptable to use abelian $1$-nilpotent $p$-quotients.

Suppose the algorithm returns a value within $1/4$ of an integer (as we have seen, this happens frequently).\
If the error is $<3/4$, then $\chi^{(2)}(\tilde M; \phi)$ is exactly `round(twisted_l2.characteristic(...))`.

Heuristically:
- the error bound is of the form $C \cdot |L|^{-1/2}$
- we can take $C = 20$ (much higher than the coefficients in the $1$-nilpotent case)

so we need $20 \cdot |L|^{-1/2} < 3/4$, or $|L| \ge 712$.
This is done by choosing e.g. $p = 29$, leading to $|L| = 29^2 = 841$.

[Note: $L\simeq \mathbb Z_{29}^2$ will be a subgroup of a finite quotient of $G$ isomorphic to $\mathbb Z_{29}^3$. See the next cell.]

In [41]:
print("  p | finite quotient ")
print("----|-----------------")
for i in range(10):
    exps = tuple([0]*i + [1])
    _, Fin = twisted_l2.finite_quotient(G, exps)
    print(f" {Primes().unrank(i) :>2} | {Fin.StructureDescription()}")

  p | finite quotient 
----|-----------------
  2 | C2 x C2 x C2
  3 | C3 x C3 x C3
  5 | C5 x C5 x C5
  7 | C7 x C7 x C7
 11 | C11 x C11 x C11
 13 | C13 x C13 x C13
 17 | C17 x C17 x C17
 19 | C19 x C19 x C19
 23 | C23 x C23 x C23
 29 | C29 x C29 x C29


In [42]:
exps29 = (0,0,0,0,0,0,0,0,0,1)

In [44]:
%%time
for v in [(1,0,0), (0,1,0), (0,0,1)]:
    print(f"===== {v} =====")
    print(f"----- n = 1 -----")
    twisted_l2.characteristic(G, v, cc, 1, exps29)
    print(f"----- n = 2 -----")
    twisted_l2.characteristic(G, v, cc, 2, exps29)
    print(flush=True)

===== (1, 0, 0) =====
----- n = 1 -----
Degrees of Laplacians : [2.0, 5.86206897, 3.86206897]
Valuations            : [0.0, 0.06896552, 0.06896552]
Quotient sizes        : [1, 841, 841]
Characteristic        : -0.93103448
----- n = 2 -----
Degrees of Laplacians : [2.0, 5.72413793, 3.72413793]
Valuations            : [0.0, 0.13793103, 0.13793103]
Quotient sizes        : [841, 841, 841]
Characteristic        : -0.86206897

===== (0, 1, 0) =====
----- n = 1 -----
Degrees of Laplacians : [2.0, 5.86206897, 3.86206897]
Valuations            : [0.0, 0.06896552, 0.06896552]
Quotient sizes        : [1, 841, 841]
Characteristic        : -0.93103448
----- n = 2 -----
Degrees of Laplacians : [2.0, 5.72413793, 3.72413793]
Valuations            : [0.0, 0.13793103, 0.13793103]
Quotient sizes        : [841, 841, 841]
Characteristic        : -0.86206897

===== (0, 0, 1) =====
----- n = 1 -----
Degrees of Laplacians : [2.0, 5.86206897, 3.86206897]
Valuations            : [0.0, 0.06896552, 0.06896552]
Qu

For all three points, the valuations stabilize at $[0,0,0]$ when increasing `n`. So, in hindsight, `n = 1` suffices.\
Since the Thurston norm is an even function, we infer _six_ values:

- $(\pm 1, \phantom{\pm} 0, \phantom{\pm} 0) \mapsto -1$
- $(\phantom{\pm} 0, \pm 1, \phantom{\pm} 0) \mapsto -1$
- $(\phantom{\pm} 0, \phantom{\pm} 0, \pm 1) \mapsto -1$

We try $4$ more values of $\phi$:

In [46]:
%%time
for v in [(1,1,1), (-1,1,1), (1,-1,1), (1,1,-1)]:
    print(f"===== {v} =====")
    for j in (1,2,3):
        print(f"----- n = {j} -----")
        twisted_l2.characteristic(G, v, cc, j, exps29)
    print(flush=True)

===== (1, 1, 1) =====
----- n = 1 -----
Degrees of Laplacians : [2.0, 10.0, 8.0]
Valuations            : [0.0, 1.0, 0.0]
Quotient sizes        : [841, 841, 841]
Characteristic        : -3.0
----- n = 2 -----
Degrees of Laplacians : [2.0, 10.0, 8.0]
Valuations            : [0.0, 1.0, 0.0]
Quotient sizes        : [841, 841, 841]
Characteristic        : -3.0
----- n = 3 -----
Degrees of Laplacians : [2.0, 10.0, 8.0]
Valuations            : [0.0, 1.0, 0.0]
Quotient sizes        : [841, 841, 841]
Characteristic        : -3.0

===== (-1, 1, 1) =====
----- n = 1 -----
Degrees of Laplacians : [2.0, 12.0, 8.0]
Valuations            : [0.0, 1.0, 0.0]
Quotient sizes        : [841, 841, 841]
Characteristic        : -2.0
----- n = 2 -----
Degrees of Laplacians : [2.0, 10.0, 8.0]
Valuations            : [0.0, 2.0, 0.0]
Quotient sizes        : [841, 841, 841]
Characteristic        : -3.0
----- n = 3 -----
Degrees of Laplacians : [2.0, 10.0, 8.0]
Valuations            : [0.0, 2.0, 0.0]
Quotient sizes 

Again, the valuations stabilize, so we get the _eight_ values

- $(\pm 1, \pm 1, \pm 1) \mapsto -3$

The Thurston norm (which is $-\chi^{(2)}(\dots)$) is determined by these $14$ values.

Finally, we can double check the $19$ nontrivial $\phi \in \{-1,0,1\}^3$ that we didn't explicitly compute:

In [48]:
%%time

n2 = [(-1, 0, 0), ( 0,-1, 0), ( 0, 0,-1), ( 1, 1, 0), ( 1, 0, 1),
      ( 1,-1, 0), (-1, 1, 0), ( 0, 1,-1), (-1,-1, 0), ( 1, 0,-1),
      (-1, 0, 1), (-1, 0,-1), ( 0,-1, 1), ( 0, 1, 1), ( 0,-1,-1)]
# ^ their valuations stabilize with n = 2

n3 = [(-1,-1, 1), (-1, 1,-1), ( 1,-1,-1), (-1,-1,-1)]
# ^ their valuations stabilize with n = 3

for v in n2:
    print(f"===== {v} =====")
    for j in (1,2):
        print(f"----- n = {j} -----")
        twisted_l2.characteristic(G, v, cc, j, exps29)
    print(flush=True)

for v in n3:
    print(f"===== {v} =====")
    for j in (1,2,3):
        print(f"----- n = {j} -----")
        twisted_l2.characteristic(G, v, cc, j, exps29)
    print(flush=True)

===== (-1, 0, 0) =====
----- n = 1 -----
Degrees of Laplacians : [2.0, 5.86206897, 3.86206897]
Valuations            : [0.0, 0.06896552, 0.06896552]
Quotient sizes        : [1, 841, 841]
Characteristic        : -0.93103448
----- n = 2 -----
Degrees of Laplacians : [2.0, 5.72413793, 3.72413793]
Valuations            : [0.0, 0.13793103, 0.13793103]
Quotient sizes        : [841, 841, 841]
Characteristic        : -0.86206897

===== (0, -1, 0) =====
----- n = 1 -----
Degrees of Laplacians : [2.0, 5.86206897, 3.86206897]
Valuations            : [0.0, 0.06896552, 0.06896552]
Quotient sizes        : [1, 841, 841]
Characteristic        : -0.93103448
----- n = 2 -----
Degrees of Laplacians : [2.0, 5.72413793, 3.72413793]
Valuations            : [0.0, 0.13793103, 0.13793103]
Quotient sizes        : [841, 841, 841]
Characteristic        : -0.86206897

===== (0, 0, -1) =====
----- n = 1 -----
Degrees of Laplacians : [2.0, 5.86206897, 3.86206897]
Valuations            : [0.0, 0.06896552, 0.06896552]